In [1]:
__author__ = 'ZFTurbo: https://kaggle.com/zfturbo'

import numpy as np
np.random.seed(2016)
import sys
sys.path.insert(0, '/home/ubuntu/fish/scripts')
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import log_loss
from keras import __version__ as keras_version

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (sz1, sz2), cv2.INTER_LINEAR)
    return resized


def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    folders = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join('..', 'data', 'fish', 'crop', 'train', fld, '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl)
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index)
        path = os.path.join('..', 'data', 'fish', 'crop', 'valid', fld, '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl)
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index)

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id


def load_test():
    path = os.path.join('..', 'data', 'fish', 'crop', 'test', 'test', '*.jpg')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        X_test.append(img)
        X_test_id.append(flbase)

    return X_test, X_test_id


def create_submission(predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])
    result1.loc[:, 'image'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    result1.to_csv(sub_file, index=False)


def read_and_normalize_train_data():
    train_data, train_target, train_id = load_train()

    print('Convert to numpy...')
    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)

    print('Reshape...')
    train_data = train_data.transpose((0, 3, 1, 2))

    print('Convert to float...')
    train_data = train_data.astype('float32')
    train_data = train_data / 255
    train_target = np_utils.to_categorical(train_target, 8)

    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, train_id


def read_and_normalize_test_data():
    start_time = time.time()
    test_data, test_id = load_test()

    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.transpose((0, 3, 1, 2))

    test_data = test_data.astype('float32')
    test_data = test_data / 255

    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    print('Read and process test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return test_data, test_id


def dict_to_list(d):
    ret = []
    for i in d.items():
        ret.append(i[1])
    return ret


def merge_several_folds_mean(data, nfolds):
    a = np.array(data[0])
    for i in range(1, nfolds):
        a += np.array(data[i])
    a /= nfolds
    return a.tolist()


def get_lrg_layers(conv_layers):
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        Dropout(p),
        #MaxPooling2D(),
        #Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        #BatchNormalization(axis=1),
        #MaxPooling2D(),
        #Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        #BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(8,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]


def get_validation_predictions(train_data, predictions_valid):
    pv = []
    for i in range(len(train_data)):
        pv.append(predictions_valid[i])
    return pv

def run_cross_validation_create_models(nfolds=10):
    # input image dimensions
    batch_size = 32
    nb_epoch = maxepoch
    random_state = 51

    train_data, train_target, train_id = read_and_normalize_train_data()
    
    # Set up VGG weights
    vgg640 = Vgg16BN((sz1, sz2)).model
    vgg640.pop()
    vgg640.input_shape, vgg640.output_shape
    vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])
    conv_layers,_ = split_at(vgg640, Convolution2D)
    conv_trn_feat = vgg640.predict(train_data, batch_size=32, verbose=1)
    
    
    yfull_train = dict()
    # kf = KFold(len(train_id), n_folds=nfolds, shuffle=True, random_state=random_state)
    valset = pd.read_csv("../image_validation_set.csv")
    kf = KFold(int(valset.Cluster.nunique()), n_folds=nfolds, shuffle = True, random_state=random_state)
    num_fold = 0
    sum_score = 0
    models = []
    for train_index, test_index in kf:
        model = Sequential(get_lrg_layers(conv_layers))
        model.compile(Adam(lr=lrate), loss='categorical_crossentropy', metrics=['accuracy'])
        teid = [i for i in valset.file_name.values if valset[valset['file_name']==i]['Cluster'].values[0] in test_index]
        trid = [i for i in valset.file_name.values if valset[valset['file_name']==i]['Cluster'].values[0] in train_index]
        test_index1 = [i for i in range(len(train_id)) if train_id[i] in teid]
        train_index1 = [i for i in range(len(train_id)) if train_id[i] in trid]
        X_train = conv_trn_feat[train_index1]
        Y_train = train_target[train_index1]
        X_valid = conv_trn_feat[test_index1]
        Y_valid = train_target[test_index1]

        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, verbose=1),
        ]
        model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
              shuffle=False, verbose=1, validation_data=(X_valid, Y_valid),
              callbacks=callbacks)

        predictions_valid = model.predict(X_valid.astype('float32'), batch_size=batch_size, verbose=2)
        score = log_loss(Y_valid, predictions_valid)
        print('Score log_loss: ', score)
        sum_score += score*len(test_index)

        # Store valid predictions
        for i in range(len(test_index)):
            yfull_train[test_index[i]] = predictions_valid[i]

        models.append(model)

    score = sum_score/len(train_data)
    print("Log_loss train independent avg: ", score)

    info_string = 'loss_' + str(score) + '_folds_' + str(nfolds) + '_ep_' + str(nb_epoch)
    return info_string, models


def run_cross_validation_process_test(info_string, models):
    batch_size = 16
    num_fold = 0
    yfull_test = []
    test_id = []
    nfolds = len(models)

    for i in range(nfolds):
        model = models[i]
        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        test_data, test_id = read_and_normalize_test_data()
        test_prediction = model.predict(test_data, batch_size=batch_size, verbose=2)
        yfull_test.append(test_prediction)

    test_res = merge_several_folds_mean(yfull_test, nfolds)
    info_string = 'loss_' + info_string \
                + '_folds_' + str(nfolds)
    create_submission(test_res, test_id, info_string)

In [3]:
sz1, sz2 = 256, 256
nf = 128
p = 0.3
lrate = 1e-4
maxepoch = 300 

In [4]:
if __name__ == '__main__':
    print('Keras version: {}'.format(keras_version))
    num_folds = 5
    info_string, models = run_cross_validation_create_models(num_folds)
    run_cross_validation_process_test(info_string, models)

Keras version: 1.2.2
Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder NoF (Index: 4)
Load folder OTHER (Index: 5)
Load folder SHARK (Index: 6)
Load folder YFT (Index: 7)
Read train data time: 15.96 seconds
Convert to numpy...
Reshape...
Convert to float...
Train shape: (3307, 3, 256, 256)
3307 train samples
3307/3307 [==============================] - 102s   
Start KFold number 1 from 5
Split train:  2586 2586
Split valid:  721 721
Train on 2586 samples, validate on 721 samples
Epoch 1/300
2586/2586 [==============================] - 3s - loss: 2.0350 - acc: 0.4524 - val_loss: 2.0660 - val_acc: 0.0527
Epoch 2/300
2586/2586 [==============================] - 3s - loss: 1.7232 - acc: 0.4362 - val_loss: 1.9814 - val_acc: 0.2316
Epoch 3/300
2586/2586 [==============================] - 3s - loss: 1.6972 - acc: 0.3650 - val_loss: 1.9080 - val_acc: 0.3370
Epoch 4/300
2586/2586 [===========================

ValueError: Error when checking : expected batchnormalization_input_1 to have shape (None, 512, 16, 16) but got array with shape (694, 3, 256, 256)